# Training and deploying a tabular model using Vertex custom training job - Part 2

![Training pipeline](../images/custom-tabular.png)

In [13]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time

import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions

## Configure GCP settings

*Before running the notebook make sure to follow the repo's README file to install the pre-requisites.*

In [14]:
PROJECT = 'jk-vertex-workshop'
REGION = 'us-central1'
PREFIX = 'jk1'

STAGING_BUCKET = f'gs://{PREFIX}-bucket'
VERTEX_SA = f'training-sa@{PROJECT}.iam.gserviceaccount.com'

TENSORBOARD = 'projects/910094146258/locations/us-central1/tensorboards/6406405654306619392'

BQ_DATASET_NAME = f'{PREFIX}_dataset' 
BQ_TRAIN_SPLIT_NAME = 'training'
BQ_VALID_SPLIT_NAME = 'validation'
BQ_TEST_SPLIT_NAME = 'testing'
BQ_LOCATION = 'US'

## Deploying a model to Vertex AI

### Initialize Vertex AI SDK

In [15]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### List custom jobs

In [33]:
search_filter = 'display_name="{}_CUSTOM_SCRIPT"'.format(PREFIX)

for job in vertex_ai.CustomJob.list(filter=search_filter, order_by='create_timedesc'):
    print(job.display_name, job.resource_name, job.state, job.create_time)

jk1_CUSTOM_SCRIPT projects/910094146258/locations/us-central1/customJobs/2295819861214887936 JobState.JOB_STATE_RUNNING 2021-06-22 03:20:01.916193+00:00
jk1_CUSTOM_SCRIPT projects/910094146258/locations/us-central1/customJobs/8617184888182800384 JobState.JOB_STATE_RUNNING 2021-06-22 03:09:45.985600+00:00


### Wait for the oldest custom job to complete

In [ ]:
while True:
    job_state = job.state
    if job_state != vertex_ai.gapic.JobState.JOB_STATE_SUCCEEDED:
        print("Job has not completed:", job_state)
        if (job_state == vertex_ai.gapic.JobState.JOB_STATE_FAILED or 
           job_state == vertex_ai.gapic.JobState.JOB_STATE_CANCELLED):
            break
    else:
        print("Job has completed")
        break
    time.sleep(60)

Job has not completed: JobState.JOB_STATE_RUNNING


In [24]:
model_artifacts = job.job_spec.base_output_directory.output_uri_prefix 

In [32]:
job.state

<JobState.JOB_STATE_RUNNING: 3>

In [25]:
!gsutil ls {model_artifacts}

gs://jk1-bucket/jobs/jk1_CUSTOM_SCRIPT/20210622_030944/aiplatform-2021-06-22-03:09:45.367-aiplatform_custom_trainer_script-0.1.tar.gz
gs://jk1-bucket/jobs/jk1_CUSTOM_SCRIPT/20210622_030944/logs/


### Inspect the model

In [ ]:
saved_model_path = f'{best_model_dir}/model'
#saved_model_path = 'gs://jk-vertex-workshop-bucket/models/taxi'

!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

### Upload the model

In [ ]:
display_name = f'{PREFIX} Chicago taxi tip classifier'
description = 'Chicago taxi tip TensorFlow classifier'
serving_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

### Create an endpoint

In [ ]:
display_name = f'{PREFIX} Taxi tip classifier endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)

### Deploy the Model

In [ ]:
deployed_model_display_name = f'{PREFIX}-taxi-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


## Invoking the deployed model using Vertex SDK

### Get the endpoint

In [ ]:
filter = f'display_name="{PREFIX} Taxi tip classifier endpoint"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

### Call the endpoint

In [ ]:
test_instances = [  
    
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

## Clean up

### Undeploy models

In [ ]:
endpoint.list_models()

In [ ]:
endpoint.undeploy_all()

### Delete endpoint

In [ ]:
endpoint.delete()

### Delete model

In [ ]:
model.delete()